# Define the level 1 models

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import yaml
from os.path import join
import random
import data_prep

# Read in config file
hyperparams = yaml.safe_load(open('hyperparams.yaml'))
print(hyperparams)

# Hyperparameters
num_features = hyperparams['num_features']  # Number of features
use_time_horizon = hyperparams['use_time_horizon']  # Use time horizon
HORIZON = hyperparams['horizon']  # Number of days into the future to predict
DAYS_FORWARD = hyperparams['days_forward']  # Number of days into the future to predict
END_SPLIT = hyperparams['end_split']  # End of the split
DATA_PATH = join('data', 'new_dataset', 'Finalised_datasets', 'source_price_AMZN_TH.csv')#join('data', 'original_dataset', 'Finalised_datasets', 'amzn_all_sources_WITH_TH_2017-2020.csv') #'Finalised_datasets',
models = hyperparams['models']  # Models to train

print(models)
lstm_params = models[0]

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(num_features, lstm_params['hidden_size'], batch_first=True)
        self.dropout1 = nn.Dropout(lstm_params['dropout'])
        self.lstm2 = nn.LSTM(lstm_params['hidden_size'], lstm_params['hidden_size'], batch_first=True)
        self.dropout2 = nn.Dropout(lstm_params['dropout'])
        self.lstm3 = nn.LSTM(lstm_params['hidden_size'],lstm_params['hidden_size'], batch_first=True)
        self.dropout3 = nn.Dropout(lstm_params['dropout'])
        self.lstm4 = nn.LSTM(lstm_params['hidden_size'],lstm_params['hidden_size'], batch_first=True)  
        self.dropout4 = nn.Dropout(lstm_params['dropout'])
        self.fc = nn.Linear(lstm_params['hidden_size'], 1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if use_time_horizon:
            outputs = []
            # For each sequence in the batch
            for i in range(x.shape[0]):
                x_ele = x[i]
                # remove padding
                x_ele = x_ele[x_ele[:, 0] != -1]
                # print("before", x_ele.shape)
                x_ele = x_ele.unsqueeze(0) # Add a batch dimension
                # print("after", x_ele.shape)

                out, _ = self.lstm1(x_ele)  # process single sequence
                out = self.dropout1(out)
                out, _ = self.lstm2(out)
                out = self.dropout2(out)
                out, _ = self.lstm3(out)
                out = self.dropout3(out)
                out, _ = self.lstm4(out)
                out = out[:, -1, :]  # take the last output from the last LSTM layer
                out = self.dropout4(out)
                out = self.fc(out)
                out = self.sigmoid(out)
                outputs.append(out)

            outputs = torch.cat(outputs, dim=0)  # recombine into a single batch tensor
            return outputs
        else:
            # print("x shape", x.shape)
            x, _ = self.lstm1(x)
            x = self.dropout1(x)
            x, _ = self.lstm2(x)
            x = self.dropout2(x)
            x, _ = self.lstm3(x)
            x = self.dropout3(x)
            x, _ = self.lstm4(x)
            x = x[:, -1, :]  # Take the last output from the last LSTM layer
            x = self.dropout4(x)
            x = self.fc(x)   # Linear layer to map to 1 output
            x = self.sigmoid(x)
            return x

gru_params = models[1]
# GRU Model
class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(num_features, gru_params['hidden_size'], batch_first=True)
        self.dropout1 = nn.Dropout(gru_params['dropout'])
        self.gru2 = nn.GRU(gru_params['hidden_size'], gru_params['hidden_size'], batch_first=True)
        self.dropout2 = nn.Dropout(gru_params['dropout'])
        self.gru3 = nn.GRU(gru_params['hidden_size'], gru_params['hidden_size'], batch_first=True)
        self.dropout3 = nn.Dropout(gru_params['dropout'])
        self.gru4 = nn.GRU(gru_params['hidden_size'], gru_params['hidden_size'], batch_first=True)
        self.dropout4 = nn.Dropout(gru_params['dropout'])
        self.fc = nn.Linear(gru_params['hidden_size'], 1)  # Ensures the output is of size [batch_size, 1]
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        if use_time_horizon:

            outputs = []
            # For each sequence in the batch
            for i in range(x.shape[0]):
                x_ele = x[i]
                # remove padding
                x_ele = x_ele[x_ele[:, 0] != -1]
                x_ele = x_ele.unsqueeze(0)
                # print(x_ele.shape)

                # Pass through the GRU layers
                x_ele, _ = self.gru1(x_ele)
                x_ele = self.dropout1(x_ele)
                x_ele, _ = self.gru2(x_ele)
                x_ele = self.dropout2(x_ele)
                x_ele, _ = self.gru3(x_ele)
                x_ele = self.dropout3(x_ele)
                x_ele, _ = self.gru4(x_ele)
                x_ele = x_ele[:, -1, :]  # Take the last output
                x_ele = self.dropout4(x_ele)
                x_ele = self.fc(x_ele)   # Linear layer to map to 1 output
                x_ele = self.sigmoid(x_ele)
                outputs.append(x_ele)

            outputs = torch.cat(outputs, dim=0)  # recombine into a single batch tensor
            return outputs
        else:
            x, _ = self.gru1(x)
            x = self.dropout1(x)
            x, _ = self.gru2(x)
            x = self.dropout2(x)
            x, _ = self.gru3(x)
            x = self.dropout3(x)
            x, _ = self.gru4(x)
            x = x[:, -1, :]  # Take the last output
            x = self.dropout4(x)
            x = self.fc(x)   # Linear layer to map to 1 output
            x = self.sigmoid(x)
            return x
        
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

{'num_features': 11, 'use_time_horizon': True, 'horizon': 1, 'days_forward': 1, 'end_split': 50, 'return_lowest_val_loss': False, 'models': [{'model': 'lstm', 'hidden_size': 60, 'dropout': 0.2, 'learning_rate': 0.002, 'batch_size': 16, 'num_epochs': 250, 'shuffle': True}, {'model': 'gru', 'hidden_size': 50, 'dropout': 0.4, 'learning_rate': 0.0008, 'batch_size': 16, 'num_epochs': 150, 'shuffle': True}, {'model': 'mlp', 'hidden_size': 25, 'learning_rate': 0.002, 'batch_size': 2, 'num_epochs': 750, 'shuffle': False}]}
[{'model': 'lstm', 'hidden_size': 60, 'dropout': 0.2, 'learning_rate': 0.002, 'batch_size': 16, 'num_epochs': 250, 'shuffle': True}, {'model': 'gru', 'hidden_size': 50, 'dropout': 0.4, 'learning_rate': 0.0008, 'batch_size': 16, 'num_epochs': 150, 'shuffle': True}, {'model': 'mlp', 'hidden_size': 25, 'learning_rate': 0.002, 'batch_size': 2, 'num_epochs': 750, 'shuffle': False}]


# Load in data

In [2]:
set_seed(42)
import pandas as pd
import numpy as np

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, TensorDataset


def get_data_stanford():

    # To drop columns, change the data_prep.py file

    # split_y: tuple
    #     (train_y, validate_y, test_y)
    # split_X: tuple
    #     (train_X, validate_X, test_X)
    split_y, split_X = data_prep.data_prep(DATA_PATH, HORIZON, DAYS_FORWARD, END_SPLIT, use_time_horizon)


    return split_X, split_y



(x_train, x_val, x_test), (y_train, y_val, y_test) = get_data_stanford()

print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)
print('x_val.shape', x_val.shape)
print('y_val.shape', y_val.shape)
print('x_test.shape', x_test.shape)
print('y_test.shape', y_test.shape)




    mean compound     TH  mean compound twitter     Volume        Open  \
29       0.229475   7.50               0.153945  2792400.0  837.000000   
30       0.061900   5.55               0.173452  2968900.0  834.000000   
31       0.177900   6.25               0.180554  2714700.0  841.840027   
32       0.430975  16.75               0.173320  3112300.0  842.000000   
33       0.443750   8.25               0.160674  3507700.0  848.840027   

          High         Low   Adj Close  SMA_Indicator  Bollinger_Indicator  \
29  838.309998  831.450012  836.390015              1                    2   
30  842.809998  832.820007  842.700012              1                    2   
31  845.000000  839.380005  844.140015              1                    2   
32  847.270020  840.729980  845.070007              1                    2   
33  857.979980  847.250000  856.440002              1                    2   

    Close_diff_UBB  Close_diff_LBB  
29      -10.773347       36.097365  
30       -6.

# Instatiate the models

In [3]:
set_seed(42)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# Instantiate models
lstm_model = LSTMModel()
gru_model = GRUModel()


# Define loss and optimizer
criterion = nn.BCELoss()
lstm_optimizer = optim.RMSprop(lstm_model.parameters(), lr=lstm_params['learning_rate'], weight_decay=1e-5) # 16 batch size, 150 epochs
gru_optimizer = optim.RMSprop(gru_model.parameters(), lr=gru_params['learning_rate'], weight_decay=1e-5) # 16 batch size, 200 epochs


base_models_batch_size = lstm_params['batch_size'] # same batch size for both models

# Train the base models

In [4]:
set_seed(42)
import copy

# Convert data to PyTorch tensors and create DataLoader
X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=base_models_batch_size, shuffle=lstm_params['shuffle']) #Stanford had shuffle true

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=base_models_batch_size) 

import torch

def train_model(model, optimizer, criterion, train_loader, n_epochs=150, val_loader=None, return_lowest_val_loss=False, scheduler=None):
    model.train()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    best_val_accuracy = 0

    for epoch in range(n_epochs):
        epoch_loss = 0
        correct_train = 0
        total_train = 0
        
        # Training phase
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch.view(-1, 1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            # Calculate training accuracy
            predicted_train = output.round()
            total_train += y_batch.size(0)
            correct_train += (predicted_train == y_batch.view(-1, 1)).sum().item()

        # Output training loss and accuracy
        train_loss_avg = epoch_loss / len(train_loader)
        train_accuracy = correct_train / total_train
        print(f'Epoch {epoch+1}/{n_epochs}, Training Loss: {train_loss_avg:.4f}, Training Accuracy: {train_accuracy:.2f}', end='')

        # Validation phase (if val_loader is provided)
        if val_loader:
            model.eval()
            val_loss = 0
            correct_val = 0
            total_val = 0
            with torch.no_grad():
                for X_val, y_val in val_loader:
                    output_val = model(X_val)
                    val_loss += criterion(output_val, y_val.view(-1, 1)).item()

                    predicted_val = output_val.round()
                    total_val += y_val.size(0)
                    correct_val += (predicted_val == y_val.view(-1, 1)).sum().item()

            val_loss_avg = val_loss / len(val_loader)
            val_accuracy = correct_val / total_val
            print(f', Validation Loss: {val_loss_avg:.4f}, Validation Accuracy: {val_accuracy:.2f}')
            
            # if val_accuracy > best_val_accuracy - 1e-2:
            #     best_val_loss = val_loss_avg
            #     best_val_accuracy = val_accuracy
            #     best_model_wts = copy.deepcopy(model.state_dict())
            # # Check if this is the best model so far
            if val_loss_avg < best_val_loss + 1e-2 : # Add 1e-2 as a "buffer" to favor the latest model
                best_val_loss = val_loss_avg
                best_val_accuracy = val_accuracy
                best_model_wts = copy.deepcopy(model.state_dict())

            model.train()

        else:
            print()  # Just move to the next line

        # Step the scheduler
        if scheduler is not None:
            if scheduler.__class__.__name__ == 'ReduceLROnPlateau':
                scheduler.step(val_loss_avg)
            else:
                scheduler.step()

    # Load the best model weights
    if return_lowest_val_loss and val_loader != None:
        print(f'Lowest validation loss: {best_val_loss:.4f}')
        print(f'Best validation accuracy: {best_val_accuracy:.2f}')
        model.load_state_dict(best_model_wts)
    return model

# Train the LSTM model
print("Training LSTM Model")
train_model(lstm_model, lstm_optimizer, criterion, train_loader, lstm_params['num_epochs'], val_loader, hyperparams['return_lowest_val_loss'])


# Train the GRU model
print("Training GRU Model")
train_model(gru_model, gru_optimizer, criterion, train_loader, gru_params['num_epochs'], val_loader, hyperparams['return_lowest_val_loss']) # 200

torch.Size([624, 26, 11])
torch.Size([624])
Training LSTM Model
Epoch 1/250, Training Loss: 0.6935, Training Accuracy: 0.55, Validation Loss: 0.6945, Validation Accuracy: 0.50
Epoch 2/250, Training Loss: 0.6875, Training Accuracy: 0.57, Validation Loss: 0.6961, Validation Accuracy: 0.50
Epoch 3/250, Training Loss: 0.6863, Training Accuracy: 0.57, Validation Loss: 0.6991, Validation Accuracy: 0.50
Epoch 4/250, Training Loss: 0.6862, Training Accuracy: 0.57, Validation Loss: 0.6996, Validation Accuracy: 0.50
Epoch 5/250, Training Loss: 0.6868, Training Accuracy: 0.57, Validation Loss: 0.7001, Validation Accuracy: 0.50
Epoch 6/250, Training Loss: 0.6854, Training Accuracy: 0.57, Validation Loss: 0.6978, Validation Accuracy: 0.50
Epoch 7/250, Training Loss: 0.6860, Training Accuracy: 0.57, Validation Loss: 0.6996, Validation Accuracy: 0.50
Epoch 8/250, Training Loss: 0.6855, Training Accuracy: 0.57, Validation Loss: 0.7008, Validation Accuracy: 0.50
Epoch 9/250, Training Loss: 0.6861, Trai

GRUModel(
  (gru1): GRU(11, 50, batch_first=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (gru2): GRU(50, 50, batch_first=True)
  (dropout2): Dropout(p=0.4, inplace=False)
  (gru3): GRU(50, 50, batch_first=True)
  (dropout3): Dropout(p=0.4, inplace=False)
  (gru4): GRU(50, 50, batch_first=True)
  (dropout4): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

# Use base models to predict the validation data, this will be used as input to the Meta Learner

In [5]:
set_seed(42)
lstm_val_predictions = lstm_model(torch.tensor(x_val, dtype=torch.float32)).detach().numpy().reshape(-1,1)
gru_val_predictions = gru_model(torch.tensor(x_val, dtype=torch.float32)).detach().numpy().reshape(-1,1)

# lstm_pred = lstm_model.predict(X).reshape(-1, 1)
# gru_pred = gru_model.predict(X).reshape(-1, 1)

# Form and return new data set
# new_X = np.hstack((lstm_pred, gru_pred))


# Combine predictions to form new training data for the meta-learner
meta_X_train = np.concatenate((lstm_val_predictions, gru_val_predictions), axis=1)#meta_X_train = np.hstack((lstm_val_predictions, gru_val_predictions))#

print(meta_X_train.shape)



(50, 2)


# Define the meta learner

In [6]:
set_seed(42)
# it's a fully-connect neuralnetwork with three layers; the activation function for this model is the Rectified Linear Unit (ReLu).
# NOTE: The paper doesn't specify the number of neurons in the hidden layers, so I'm basing on the stanford paper

hyperparams = yaml.safe_load(open('hyperparams.yaml'))
models = hyperparams['models']
meta_params = models[2]

class MetaLearner(nn.Module):
    # def __init__(self):
    #     super(MetaLearner, self).__init__()
    #     self.fc1 = nn.Linear(2, meta_params['hidden_size'], bias=True)
    #     self.fc12 = nn.Linear(meta_params['hidden_size'], 1, bias=True)
    #     self.sigmoid = nn.Sigmoid() 
    #     self.relu = nn.ReLU()

    def __init__(self):
        super(MetaLearner, self).__init__()
        self.fc1 = nn.Linear(2, meta_params['hidden_size'], bias=True)
        # self.fc12 = nn.Linear(meta_params['hidden_size'], 1, bias=False)
        self.fc2 = nn.Linear(meta_params['hidden_size'], meta_params['hidden_size'])
        self.fc3 = nn.Linear(meta_params['hidden_size'], meta_params['hidden_size'])
        self.fc4 = nn.Linear(meta_params['hidden_size'], 1)
        self.sigmoid = nn.Sigmoid() 
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        # x = self.fc12(x)
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x) 
        return x
    
lstm_test_predictions = lstm_model(torch.tensor(x_test, dtype=torch.float32)).detach()
gru_test_predictions = gru_model(torch.tensor(x_test, dtype=torch.float32)).detach()

# Train meta learner

In [7]:
set_seed(42)
meta_model = MetaLearner()
meta_criterion = nn.BCELoss()
meta_optimizer = optim.Adam(meta_model.parameters(), lr=meta_params['learning_rate'])
# scheduler = torch.optim.lr_scheduler.StepLR(meta_optimizer, step_size=100, gamma=0.1)

meta_learner_batch_size = meta_params['batch_size']

meta_X_train_tensor = torch.tensor(meta_X_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

meta_train_dataset = TensorDataset(meta_X_train_tensor, y_val_tensor)
meta_train_loader = DataLoader(meta_train_dataset, batch_size=meta_learner_batch_size, shuffle=meta_params['shuffle'])





# TODO : remove after submitting
meta_X_test = np.concatenate((lstm_test_predictions, gru_test_predictions), axis=1)
meta_X_test_tensor = torch.tensor(meta_X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

val_dataset = TensorDataset(meta_X_test_tensor, y_test_tensor)
val_loader = DataLoader(val_dataset, batch_size=meta_params['batch_size']) 


train_model(meta_model, meta_optimizer, meta_criterion, meta_train_loader, meta_params['num_epochs'], val_loader, return_lowest_val_loss=False)

Epoch 1/750, Training Loss: 0.6953, Training Accuracy: 0.46, Validation Loss: 0.6933, Validation Accuracy: 0.50
Epoch 2/750, Training Loss: 0.6942, Training Accuracy: 0.50, Validation Loss: 0.6933, Validation Accuracy: 0.50
Epoch 3/750, Training Loss: 0.6941, Training Accuracy: 0.50, Validation Loss: 0.6932, Validation Accuracy: 0.50
Epoch 4/750, Training Loss: 0.6941, Training Accuracy: 0.44, Validation Loss: 0.6932, Validation Accuracy: 0.50
Epoch 5/750, Training Loss: 0.6940, Training Accuracy: 0.42, Validation Loss: 0.6932, Validation Accuracy: 0.50
Epoch 6/750, Training Loss: 0.6940, Training Accuracy: 0.40, Validation Loss: 0.6932, Validation Accuracy: 0.50
Epoch 7/750, Training Loss: 0.6939, Training Accuracy: 0.38, Validation Loss: 0.6931, Validation Accuracy: 0.50
Epoch 8/750, Training Loss: 0.6939, Training Accuracy: 0.40, Validation Loss: 0.6931, Validation Accuracy: 0.50
Epoch 9/750, Training Loss: 0.6938, Training Accuracy: 0.44, Validation Loss: 0.6930, Validation Accurac

MetaLearner(
  (fc1): Linear(in_features=2, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=25, bias=True)
  (fc4): Linear(in_features=25, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)

In [8]:
set_seed(42)
from sklearn.metrics import precision_recall_fscore_support

meta_test_predictions = meta_model(meta_X_test_tensor).detach()
# Convert predictions to binary by rounding
predicted_test = meta_test_predictions.round()

print(predicted_test.numpy().flatten())

# Calculate accuracy
total_test = y_test.shape[0]
correct_test = (predicted_test.numpy().flatten() == y_test).sum()
accuracy = correct_test / total_test

# Print the accuracy
print(f'Accuracy: {accuracy:.4f}')

precision, recall, f1, _ = precision_recall_fscore_support(y_test, predicted_test.numpy().flatten(), average='binary')
print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1}')


predicted_test = lstm_test_predictions.round()

# Calculate accuracy
total_test = y_test.shape[0]
correct_test = (predicted_test.numpy().flatten() == y_test).sum()
accuracy = correct_test / total_test

# Print the accuracy
print(f'Accuracy for lstm: {accuracy:.4f}')
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predicted_test.flatten(), average='binary')
print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

predicted_test = gru_test_predictions.round()

# Calculate accuracy
total_test = y_test.shape[0]
correct_test = (predicted_test.numpy().flatten() == y_test).sum()
accuracy = correct_test / total_test

# Print the accuracy
print(f'Accuracy for gru: {accuracy:.4f}')
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predicted_test.flatten(), average='binary')
print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1}')



[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0.]
Accuracy: 0.6400
Precision: 0.7333333333333333, Recall: 0.44, F1 Score: 0.55
Accuracy for lstm: 0.5000
Precision: 0.5, Recall: 1.0, F1 Score: 0.6666666666666666
Accuracy for gru: 0.5600
Precision: 0.5384615384615384, Recall: 0.84, F1 Score: 0.65625


In [10]:
#Save the models
save_path = join('models/')
# torch.save(lstm_model.state_dict(), save_path + 'lstm_model_amzn_dynamic_final.pth')
# torch.save(gru_model.state_dict(), save_path + 'gru_model_amzn_dynamic_final.pth')
# torch.save(meta_model.state_dict(), save_path + 'meta_model_amzn_dynamic_final.pth')

# load the models
# lstm_model = LSTMModel()
# lstm_model.load_state_dict(torch.load(save_path + 'lstm_model_amzn2.pth'))
# lstm_model.eval()

# gru_model = GRUModel()
# gru_model.load_state_dict(torch.load(save_path + 'gru_model_amzn2.pth'))
# gru_model.eval()

# meta_model = MetaLearner()
# meta_model.load_state_dict(torch.load(save_path + 'meta_model_amzn2.pth'))
# meta_model.eval()

